In [1]:
import cv2
import numpy as np

import theano
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense,Activation
from keras.optimizers import SGD, Adam,Adagrad
from sklearn.cross_validation import train_test_split

from sklearn.datasets import load_digits

Using Theano backend.
/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def resize(image, size=(12, 12)): return cv2.resize(image, size).flatten()

In [3]:
digits = load_digits(n_class=2)
classes, ids = np.unique(digits['target'], return_inverse=True)
labels = np_utils.to_categorical(ids, len(classes))

data = map(resize, digits['images'])
data = np.array(data) / 255.0
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [4]:
model = Sequential()
model.add(Dense(200, input_shape=data[0].shape, kernel_initializer="uniform", activation="tanh"))
model.add(Dense(100, kernel_initializer="uniform", activation="tanh"))
model.add(Dense(labels.shape[1]))
model.add(Activation("softmax"))

In [5]:
sgd = SGD(lr=0.01)
adagrad = Adagrad(lr=0.01)
adam = Adam(lr=0.01)

model.compile(loss="binary_crossentropy", optimizer=adagrad, metrics=["accuracy"])
model.fit(trainData, trainLabels, nb_epoch=100, batch_size=128, verbose=1)

print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy*100))

/usr/local/lib/python2.7/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
270/270 [==============================] - 0s - loss: 0.6540 - acc: 0.5444     
Epoch 2/100
270/270 [==============================] - 0s - loss: 0.4628 - acc: 0.7593     
Epoch 3/100
270/270 [==============================] - 0s - loss: 0.1205 - acc: 0.9926     
Epoch 4/100
270/270 [==============================] - 0s - loss: 0.0627 - acc: 0.9926     
Epoch 5/100
270/270 [==============================] - 0s - loss: 0.0437 - acc: 0.9926     
Epoch 6/100
270/270 [==============================] - 0s - loss: 0.0382 - acc: 0.9926     

In [6]:
import pymc3 as pm
import theano.tensor as T

In [7]:
trainLabels = classes[trainLabels.argmax(1)]
ann_input = theano.shared(trainData)
ann_output = theano.shared(trainLabels)

In [8]:
d_1 = (trainData.shape[1], 200)
d_2 = (200, 100)
d_3 = (100,)

with pm.Model() as neural_network:
    W_in_1 = pm.Normal('w_in_1', 0, sd=1, shape=d_1, testval=np.random.randn(*d_1))
    W_1_2 = pm.Normal('w_1_2', 0, sd=1, shape=d_2, testval=np.random.randn(*d_2))
    W_2_out = pm.Normal('w_2_out', 0, sd=1, shape=d_3, testval=np.random.randn(*d_3))
    
    act_1 = T.tanh(T.dot(ann_input, W_in_1))
    act_2 = T.tanh(T.dot(act_1, W_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, W_2_out))
    
    """    
    act_1 = T.nnet.relu(T.dot(ann_input, W_in_1))
    act_2 = T.nnet.relu(T.dot(act_1, W_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, W_2_out))
    
    """    
    out = pm.Bernoulli('out', act_out, observed=ann_output) # Binary classification -> Bernoulli likelihood
                                                            # what about multi-class?

/usr/local/lib/python2.7/site-packages/theano/tensor/basic.py:2124: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [9]:
from pymc3 import NUTS, sample,find_MAP,Slice,traceplot
from scipy import optimize

with neural_network:
    v_params = pm.variational.advi(n=1000)

Average ELBO = -21,745: 100%|██████████| 1000/1000 [00:14<00:00, 68.77it/s]
Finished [100%]: Average ELBO = -21,144


In [10]:
with neural_network:
    trace = pm.variational.sample_vp(v_params, draws=5001)

100%|██████████| 5001/5001 [00:17<00:00, 155.17it/s]


In [11]:
testLabels = classes[testLabels.argmax(1)]
ann_input.set_value(testData)
ann_output.set_value(testLabels)

ppc = pm.sample_ppc(trace, model=neural_network, samples=500) # Creater posterior predictive samples
pred = ppc['out'].mean(axis=0) > 0.5 # Use probability of > 0.5 to assume prediction of class 1

print('Accuracy = {}%'.format((testLabels == pred).mean() * 100))

100%|██████████| 500/500 [00:03<00:00, 126.55it/s]

Accuracy = 100.0%
